In [2]:
#imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.endpoints import leaguedashplayerbiostats
from nba_api.stats.endpoints import playerdashboardbygeneralsplits
from nba_api.stats.endpoints import playervsplayer
from nba_api.stats.endpoints import leaguedashteamstats
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamgamelog, boxscoresummaryv2, boxscoretraditionalv2
import time

In [4]:
def player_id(player_name):
    players = leaguedashplayerbiostats.LeagueDashPlayerBioStats()
    players_data = players.get_data_frames()[0]
    player_id = players_data[players_data['PLAYER_NAME'] == player_name].get('PLAYER_ID')
    return player_id

In [3]:
#function to get player gamelog based on Player Name rather than ID

def player_gamelog(player_name):
    id = player_id(player_name)
    gamelog = playergamelog.PlayerGameLog(player_id= id)
    return gamelog.get_data_frames()[0]

In [5]:
def playerVplayer(player_name, vs_player_name):
    id = player_id(player_name)
    vid = player_id(vs_player_name)
    gamelog = playervsplayer.PlayerVsPlayer(vs_player_id=vid, player_id=id)
    return gamelog.get_data_frames()

# playerVplayer('Nikola Jokic', 'Rudy Gobert')

In [6]:
# Will add a Home_Away column to the gamelog so that the ML model can use that as a variable
def homeOrAway(player_gamelog):
    player_gamelog['Home_Away'] = player_gamelog['MATCHUP'].apply(lambda x: 0 if '@' in x else 1)
    return player_gamelog

In [ ]:
def get_nba_teams():
    nba_teams = teams.get_teams()  # Fetch data
    # Filter to get only active NBA teams and return as DataFrame
    return pd.DataFrame(nba_teams)

nba_teams = get_nba_teams()
print(nba_teams[['id', 'abbreviation', 'full_name']])

In [14]:
def againstThisTeam(player_gamelog, home_team_abbrev, opp_team_abbrev):
    awaySearchString = home_team_abbrev + ' vs. ' + opp_team_abbrev
    homeSearchString = home_team_abbrev + ' @ ' + opp_team_abbrev
    newGameLog = player_gamelog.loc[player_gamelog.MATCHUP.isin([homeSearchString, awaySearchString])]
    return newGameLog

#Test
kingStats = player_gamelog('LeBron James')
print(againstThisTeam(kingStats, 'LAL', 'ATL'))

  SEASON_ID  Player_ID     Game_ID     GAME_DATE    MATCHUP WL  MIN  FGM  FGA  \
7     22024       2544  0022400334  DEC 06, 2024  LAL @ ATL  L   43   14   25   

   FG_PCT  ...  DREB  REB  AST  STL  BLK  TOV  PF  PTS  PLUS_MINUS  \
7    0.56  ...    10   10   11    2    3    4   3   39           3   

   VIDEO_AVAILABLE  
7                1  

[1 rows x 27 columns]


In [19]:
def boxStatsAgainstThisTeam(player_gamelog, home_team_abbrev, opp_team_abbrev):
    new_log = againstThisTeam(player_gamelog, home_team_abbrev, opp_team_abbrev)
    new_log = new_log.loc[:, ['GAME_DATE', 'MATCHUP', 'REB', 'AST', 'STL', 'BLK', 'PTS', 'MIN']]
    print(new_log)
#test
kingStats = player_gamelog('LeBron James')
boxStatsAgainstThisTeam(kingStats, 'LAL', 'ATL')

      GAME_DATE    MATCHUP  REB  AST  STL  BLK  PTS  MIN
7  DEC 06, 2024  LAL @ ATL   10   11    2    3   39   43


In [20]:
# Testing accuracy for different ML models on point predictions
kingStats = player_gamelog('LeBron James')
print(kingStats)
avgPTS = kingStats['PTS'].mean()
#Add Home_Away column
homeOrAway(kingStats)
# target
y = kingStats.PTS
# Features should be considered carefully, I just threw in ones I think make sense
# Obviously some of these like REB, AST, STL, etc probably don't make sense since we can't know these things before the game, although we might know what his average stats are for those categories before the game
kingStats_features = ['PTS', 'Home_Away', ]
X = kingStats[kingStats_features]
# Splitting data to test for accuracy
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=0)

# DecisionTreeRegressor test
kingStats_model = DecisionTreeRegressor()
kingStats_model.fit(train_X, train_y)
val_predictions = kingStats_model.predict(val_X)
# ~2.833 MAE, <- haven't really put anything in so would not trust that
print(mean_absolute_error(val_y, val_predictions))


   SEASON_ID  Player_ID     Game_ID     GAME_DATE      MATCHUP WL  MIN  FGM  \
0      22024       2544  0022400468  JAN 02, 2025  LAL vs. POR  W   36   15   
1      22024       2544  0022400454  DEC 31, 2024  LAL vs. CLE  L   34    9   
2      22024       2544  0022400408  DEC 25, 2024    LAL @ GSW  W   37   12   
3      22024       2544  0022400404  DEC 23, 2024  LAL vs. DET  L   35   10   
4      22024       2544  0022400376  DEC 21, 2024    LAL @ SAC  W   34   13   
5      22024       2544  0022400372  DEC 19, 2024    LAL @ SAC  W   34    8   
6      22024       2544  0022401220  DEC 15, 2024  LAL vs. MEM  W   34    7   
7      22024       2544  0022400334  DEC 06, 2024    LAL @ ATL  L   43   14   
8      22024       2544  0022400321  DEC 04, 2024    LAL @ MIA  L   29   12   
9      22024       2544  0022400318  DEC 02, 2024    LAL @ MIN  L   31    4   
10     22024       2544  0022400311  DEC 01, 2024    LAL @ UTA  W   36   12   
11     22024       2544  0022400048  NOV 29, 2024  L